<a href="https://colab.research.google.com/github/zawadalam7/Zawad_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [ ]:
1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


### 2) Common Imports & Polite Headers

In [ ]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [ ]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [3]:
import pandas as pd

def q1_read_table(html: str) -> pd.DataFrame:
  tables = pd.read_html(html)
  for table in tables:
    if table.shape[1] >= 3:
      return table
  raise ValueError("No suitable table found with >= 3 columns.")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
  df.columns = [col.strip() for col in df.columns]
  df = df.rename(columns={
      "Country": "Country",
      "Alpha-2 code": "Alpha-2",
      "Alpha-3 code": "Alpha-3",
      "Numeric": "Numeric"
  })
  df["Country"] = df["Country"].str.strip()
  df["Alpha-2"] = df["Alpha-2"].str.strip().str.upper()
  df["Alpha-3"] = df["Alpha-3"].str.strip().str.upper()
  df["Numeric"] = pd.to_numeric(df["Numeric"], errors="coerce").astype("Int64")
  df = df.dropna(subset=["Country", "Alpha-2", "Alpha-3", "Numeric"])
  return df

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
  return df.sort_values(by="Numeric", ascending=False).head(top)

## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [ ]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import re

def q2_parse_items(html: str) -> pd.DataFrame:
  soup = BeautifulSoup(html, "html.parser")
  items = soup.select("tr.athing")
  rows = []

  for item in items:
    rank_tag = item.select_one("span.rank")
    title_tag = item.select_one("a.storylink")
    subtext = item.find_next_sibling("tr").select_one("td.subtext")

    rank = rank_tag.text.strip().rstrip(".") if rank_tag else ""
    title = title_tag.text.strip() if title_tag else ""
    link = title_tag["href"].strip() if title_tag and title_tag.has_attr("href") else ""

    points_tag = subtext.select_one("span.score")
    points = points_tag.text.strip().split()[0] if points_tag else ""

    comments_tag = subtext.find_all("a")[-1]
    comments_text = comments_tag.text.strip() if comments_tag else ""
    comments = re.search(r"(\d+)", comments_text)
    comments = comments.group(1) if comments else ""

    user_tag = subtext.select_one("a.hnuser")
    user = user_tag.text.strip() if user_tag else ""

    rows.append({
        "rank": rank,
        "title": title,
        "link": link,
        "points": points,
        "comments": comments,
        "user": user
    })

  return pd.DataFrame(rows)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
  df["rank"] = pd.to_numeric(df["rank"], errors="coerce").fillna(0).astype(int)
  df["points"] = pd.to_numeric(df["points"], errors="coerce").fillna(0).astype(int)
  df["comments"] = pd.to_numeric(df["comments"], errors="coerce").fillna(0).astype(int)
  df["title"] = df["title"].fillna("")
  df["link"] = df["link"].fillna("")
  df["user"] = df["user"].fillna("")
  return df

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
  return df.sort_values(by="points", ascending=False).head(top)